## 1. 이벤트 루프 (Event Loop): 오케스트라의 지휘자

`asyncio`의 세계에서 가장 중요한 주인공은 **이벤트 루프**입니다. 마치 오케스트라의 지휘자나, 할 일 목록을 관리하는 매니저와 같습니다.

- **하는 일:** 이벤트 루프는 '해야 할 일(Job)' 목록을 가지고 있습니다. 목록에서 일을 하나 꺼내서 실행(함수 호출)시킵니다. 그 일이 끝나거나 "잠깐 쉴게요"라고 하면, 지휘자는 다시 제어권을 받아 다음 일을 꺼내서 실행합니다.
  
- **작동 방식:** 무한히 도는 큐(Queue)라고 생각하면 됩니다. 일이 있으면 처리하고, 없으면 CPU를 낭비하지 않고 똑똑하게 대기합니다.
  
- **협동이 중요해:** 만약 어떤 일이 혼자서 시간을 너무 오래 끌고(욕심쟁이), 지휘자에게 제어권을 돌려주지 않으면 다른 일들은 굶어 죽습니다. 그래서 `asyncio`는 **협력적**이어야 합니다.

### **방법 A: 작업을 예약하고 루프 실행하기**  

create_task()를 사용해 루프에 할 일을 미리 던져놓고, 그 후에 루프를 돌리는 방식입니다.

In [7]:
# Jupyter Notebook
import nest_asyncio
nest_asyncio.apply()

import asyncio

# sleep 2 sec 
async def helloWorld():
    await asyncio.sleep(1)
    print("Hello World")

# sleep 1 sec
async def loopStopAfter1Sec():
    await asyncio.sleep(0.5)
    print("Kill")
    
    # because of early kill, 
    # "Hello World" will not be printed
    loop.stop() 

loop = asyncio.new_event_loop()

# reservate tasks to be done
loop.create_task(helloWorld())
loop.create_task(loopStopAfter1Sec())

# run the loop
loop.run_forever()

Kill


### **방법 B: run_until_complete 사용 (일반적인 방식)**

특정 작업이 끝날 때까지만 루프를 돌리고 싶을 때 사용합니다.

In [ ]:
import asyncio

# Jupyter Notebook
import nest_asyncio
nest_asyncio.apply()

async def helloWorld():
    await asyncio.sleep(1)
    print("Hello World")

loop = asyncio.new_event_loop()

# only until helloWorld func stops
loop.run_until_complete(helloWorld())
loop.close()

Hello World


`loop.close()`는 식사를 마친 뒤 **설거지를 하고 가스 불을 끄는 과정**이라고 생각하시면 이해가 쉽습니다.

`run_until_complete()`가 완료되었다고 해서 이벤트 루프 자체가 완전히 정리된 것은 아니기 때문입니다. 구체적인 이유는 다음과 같습니다.

#### 1. 자원 해제 (Resource Cleanup)

이벤트 루프는 실행되는 동안 네트워크 소켓, 파일 핸들, 자식 프로세스 등 다양한 **시스템 자원**을 사용합니다.

* `run_until_complete()`는 특정 작업을 끝내고 제어권을 돌려줄 뿐입니다.
* `close()`를 호출해야만 루프가 잡고 있던 내부 큐(Queue)를 비우고, 사용 중이던 자원을 OS에 안전하게 반납합니다.

#### 2. 루프의 상태 확정

이벤트 루프는 여러 가지 상태를 가집니다 (실행 중, 정지됨, 닫힘).

* `close()`를 호출하면 루프는 **'닫힘(Closed)'** 상태가 됩니다.
* 이미 닫힌 루프에 다시 일을 시키려고 하면 에러가 발생하므로, 프로그램의 논리적 오류를 방지하는 방어막 역할을 합니다.

#### 3. 메모리 누수 방지

만약 루프를 닫지 않고 계속해서 새로운 루프를 생성(`new_event_loop`)하고 실행하기를 반복한다면, 이전에 사용했던 루프의 찌꺼기들이 메모리에 남아 **메모리 누수(Memory Leak)** 가 발생할 수 있습니다.

### 💡 현대 파이썬(3.7+)에서의 변화

요즘은 `loop.close()`를 직접 쓰는 일이 드뭅니다. 바로 `asyncio.run()` 때문입니다.

In [11]:
import asyncio

# Jupyter Notebook
import nest_asyncio
nest_asyncio.apply()

async def helloWorld():
    await asyncio.sleep(1)
    print("Hello World")

# 이 한 줄이 내부적으로 아래 과정을 모두 자동으로 수행합니다.
# 1. 새로운 루프 생성
# 2. run_until_complete() 실행
# 3. 마무리 작업 및 자원 해제 (loop.close 포함)
asyncio.run(helloWorld())

Hello World


## 2. 비동기 함수와 코루틴 (Coroutines)

일반적인 파이썬 함수와 비동기 함수는 다릅니다.

**일반 함수 (프린터기)**

In [12]:
def hello_printer():
    print("안녕, 나는 단순한 프린터야.")

hello_printer() # 이렇게 부르면 바로 실행됩니다.

안녕, 나는 단순한 프린터야.


**비동기 함수 (수다쟁이 펭귄)**

`def` 앞에 `async`를 붙이면 **코루틴 함수**가 됩니다.

- **코루틴 함수:** `async def`로 정의된 함수 자체 (설계도).
- **코루틴 객체:** 함수를 호출해서 만들어진 실체 (실행 가능한 작업). 

코루틴은 중간에 멈췄다가(일시 정지), 나중에 다시 시작(재생)할 수 있습니다. 이게 비동기의 핵심입니다!

일반 함수(`def`)는 호출하면 즉시 실행되지만,   

비동기 함수(`async def`)는 호출하는 순간 실행되는 것이 아니라  
**앞으로 실행될 작업 정보가 담긴 티켓 - 코루틴 객체**만 생성해서 반환합니다.

In [25]:
import asyncio, nest_asyncio
nest_asyncio.apply()

async def loudmouth_penguin(magic_number: int):
    print(f"나는 펭귄이야. 내 행운의 숫자는 {magic_number}지.")
    
print(loudmouth_penguin(magic_number=3))

<coroutine object loudmouth_penguin at 0x7fda517ce190>


/tmp/ipykernel_262170/2287150880.py:7: RuntimeWarning: coroutine 'loudmouth_penguin' was never awaited
  print(loudmouth_penguin(magic_number=3))


* **`print(loudmouth_penguin(3))`**:
    * 여기서 `loudmouth_penguin(3)`을 호출하면, 함수 내부의 `print`문이 실행되는 게 아니라 **코루틴 객체**가 만들어집니다.
    * 출력 결과에서 `<coroutine object loudmouth_penguin at ...>`가 나온 이유가 바로 이것입니다. 당신은 지금 "티켓" 자체를 프린트한 셈입니다.


* **`RuntimeWarning`이 뜨는 이유**:
    * 파이썬 입장에선 "어? 사용자가 비동기 작업을 하나 만들었는데, 이걸 실행(`await`)하지도 않고 그냥 버려버렸네? 실수한 거 아냐?"라고 판단하여 경고를 날리는 것입니다.

In [ ]:
import asyncio, nest_asyncio
nest_asyncio.apply()

async def loudmouth_penguin(magic_number: int):
    print(f"나는 펭귄이야. 내 행운의 숫자는 {magic_number}지.")

asyncio.run(loudmouth_penguin(magic_number=3))

async def psudoMain():
    await loudmouth_penguin(magic_number=3)

asyncio.run(psudoMain())

나는 펭귄이야. 내 행운의 숫자는 3지.


`asyncio.run()`은 전달받은 **코루틴 객체(티켓)를 이벤트 루프라는 입구에 넣고 실제로 상영**해주는 역할을 합니다.

1. `asyncio.run(...)` 호출
2. 이벤트 루프가 생성됨
3. 코루틴 객체 내부의 코드가 끝까지 실행됨 (`print`문 출력)
4. 루프가 종료됨

**배경 지식: 제너레이터(Generator)**

코루틴은 파이썬의 `yield`를 쓰는 '제너레이터'의 진화형입니다.  
제너레이터도 `next()`를 부를 때마다 실행되다가 `yield`를 만나면 멈추죠? 코루틴도 이 원리를 이용합니다.

In [30]:
def get_random_number():
    print("Hi")
    yield 1
    print("Hello")
    yield 7

# 실행하면 제너레이터 객체만 나옵니다.
generator = get_random_number()
print(next(generator)) # Hi 출력 후 1을 내놓고 멈춤
print(next(generator)) # Hello 출력 후 7을 내놓고 멈춤

Hi
1
Hello
7


## 3. 태스크 (Task): 이벤트 루프의 할 일 목록

**Task**는 코루틴을 포장해서 이벤트 루프에 "이거 실행해줘"라고 예약해둔 상태입니다.

In [ ]:
import asyncio

async def make_pasta():
    print("요리 중...")
    await asyncio.sleep(0.5) 
    print("완성! 🍝")

async def restaurant():
    print("식당 문 열었습니다.")
    
    # 1. 코루틴 객체 생성
    order = make_pasta()
    
    # 2. 태스크 생성
    task = asyncio.create_task(order)
    
    # 3. 중요!! 태스크가 끝날 때까지 기다려주기
    # 이걸 안 하면 'main' 함수가 바로 끝나버려서, 식당 문을 닫아버립니다.
    # 요리 다 되기도 전에 쫓겨나는 거죠.
    await task 

asyncio.run(restaurant())

식당 문 열었습니다.
요리 중...
완성! 🍝


### ⚠️ 주의: 가비지 컬렉션 (쓰레기 수거)의 함정

파이썬의 청소부(가비지 컬렉터)가 "어? 이거 아무도 안 쓰네?" 하고 이벤트 루프가 실행하기도 전에 지워버릴 수도 있습니다.  
(실제로는 파이썬이 최대한 막아주지만, 위험한 코드입니다.)

In [65]:
import asyncio
import gc  # 🧹 청소부 모듈

async def make_pasta():
    print("요리 시작! (오래 걸림)")
    await asyncio.sleep(0.5)
    print("요리 끝! 🍝")

async def restaurant():
    # 1. 변수 할당 없이 태스크 생성 (위험!)
    asyncio.create_task(make_pasta()) 
    
    print("주문 넣음. 🧹 청소부 강제 소환!")
    
    # 2. 여기서 강제로 청소부를 부릅니다!
    # "참조되지 않는 객체는 다 갖다 버려!"
    gc.collect() 
    
    # 3. 다른 작업 (청소부가 다녀간 뒤)
    await asyncio.sleep(1) 
    print("식당 마감")

asyncio.run(restaurant())

주문 넣음. 🧹 청소부 강제 소환!
요리 시작! (오래 걸림)
요리 끝! 🍝
식당 마감


## 4. `await`: 기다림의 미학

`await` 키워드는 "기다린다"는 뜻인데, **누구를 기다리냐**에 따라 행동이 완전히 다릅니다.

### (1) `await Task` (태스크를 기다릴 때)

- **행동:** "나는 잠깐 쉴게. 이벤트 루프야, 이 태스크가 끝날 때까지 다른 일을 처리하고 있어."
  
- **결과:** **제어권(Control)을 이벤트 루프에게 넘깁니다.** 다른 작업들이 실행될 기회를 얻습니다.

#### **"딴짓"은 `await` 하기 전에 해야 합니다!**

`create_task`는 "주방(이벤트 루프)에 주문서 넣기"이고, `await`는 "요리 나올 때까지 멍하니 기다리기"입니다.  
**우리가 할 수 있는 "딴짓(다른 작업)"은 바로 그 사이(틈)에 넣어야 합니다.**

#### 1. 방법 A: 기다리기 전에 내 할 일 하기
* 주방에 파스타 주문을 넣어놓고(create_task),  
* 파스타가 끓는 동안 내가 홀 청소를 하다가(print), 
* 나중에 파스타를 확인(await)하는 방식입니다.

In [94]:
import asyncio

async def make_pasta():
    print("   [주방] 🍝 파스타 요리 시작... (3초 소요)")
    await asyncio.sleep(3) 
    print("   [주방] 🍝 파스타 완성!")

async def restaurant():
    # 1. 태스크 예약 (주문 넣기) -> 백그라운드에서 요리 시작됨
    pasta_task = asyncio.create_task(make_pasta())

    # ----------------------------------------------------
    # 여기가 바로 "딴 짓"을 하는 공간입니다!
    # 파스타가 익어가는 동안, 이 코드는 멈추지 않고 실행됩니다.
    # ----------------------------------------------------
    print("[홀] 🧹 식탁을 닦습니다.")
    print("[홀] 🥄 수저를 세팅합니다.")
    print("[홀] 💧 물을 따릅니다.")
    
    # 3. 딴짓 다 했으니, 이제 파스타가 다 됐나 기다려봅니다.
    await pasta_task 
    print("[홀] 서빙 시작!")

asyncio.run(restaurant())

[홀] 🧹 식탁을 닦습니다.
[홀] 🥄 수저를 세팅합니다.
[홀] 💧 물을 따릅니다.
   [주방] 🍝 파스타 요리 시작... (3초 소요)
   [주방] 🍝 파스타 완성!
[홀] 서빙 시작!


#### 2. 방법 B: 친구(다른 태스크)랑 같이 일하기

가장 많이 쓰는 방식입니다. 파스타(create_task)도 시키고, 스테이크(create_task)도 시키는 것입니다.

이 경우 총 걸리는 시간은 **3초 + 5초 = 8초**가 아니라, 제일 긴 **5초**만 걸립니다. 동시에 돌아가니까요!

In [ ]:
import asyncio

async def make_pasta():
    print("   [주방] 🍝 파스타 시작 (3초)")
    await asyncio.sleep(3)
    print("   [주방] 🍝 파스타 끝")

async def make_steak():
    print("   [주방] 🥩 스테이크 시작 (5초)")
    await asyncio.sleep(5)
    print("   [주방] 🥩 스테이크 끝")

async def restaurant():
    # 1. 두 개의 주문을 연달아 넣습니다. (아직 안 기다림!)
    task1 = asyncio.create_task(make_pasta())
    task2 = asyncio.create_task(make_steak())

    print("[홀] 두 가지 요리를 주문 넣었습니다. 기다립니다...")

    # 2. 이제 두 요리가 다 될 때까지 기다립니다.
    # 파스타가 끓는 동안 -> 스테이크도 굽고 있습니다. (동시 진행)
    await task1
    await task2
    
    print("[홀] 모든 요리가 나왔습니다!")

asyncio.run(restaurant())

[홀] 두 가지 요리를 주문 넣었습니다. 기다립니다...
   [주방] 🍝 파스타 시작 (3초)
   [주방] 🥩 스테이크 시작 (5초)
[시계] 0.5초 소요됨
[시계] 1.5초 소요됨
[시계] 2.5초 소요됨
   [주방] 🍝 파스타 끝
[시계] 3.5초 소요됨
[시계] 4.5초 소요됨
   [주방] 🥩 스테이크 끝
[홀] 모든 요리가 나왔습니다!


In [86]:
import asyncio

async def make_pasta():
    print("   [주방] 🍝 파스타 시작 (3초)")
    await asyncio.sleep(3)
    print("   [주방] 🍝 파스타 끝")

async def make_steak():
    print("   [주방] 🥩 스테이크 시작 (5초)")
    await asyncio.sleep(5)
    print("   [주방] 🥩 스테이크 끝")

async def restaurant():
    # 1. 두 개의 주문을 연달아 넣습니다. (아직 안 기다림!)
    task1 = asyncio.create_task(make_pasta())
    task2 = asyncio.create_task(make_steak())

    print("[홀] 두 가지 요리를 주문 넣었습니다. 기다립니다...")

    # 2. 이제 두 요리가 다 될 때까지 기다립니다.
    # 파스타가 끓는 동안 -> 스테이크도 굽고 있습니다. (동시 진행)
    await task1
    await task2

    for t in range(5):
        await asyncio.sleep(0.5)
        print(f'[시계] {t+0.5}초 소요됨')
        await asyncio.sleep(0.5)
    
    print("[홀] 모든 요리가 나왔습니다!")

asyncio.run(restaurant())

[홀] 두 가지 요리를 주문 넣었습니다. 기다립니다...
   [주방] 🍝 파스타 시작 (3초)
   [주방] 🥩 스테이크 시작 (5초)
   [주방] 🍝 파스타 끝
   [주방] 🥩 스테이크 끝
[시계] 0.5초 소요됨
[시계] 1.5초 소요됨
[시계] 2.5초 소요됨
[시계] 3.5초 소요됨
[시계] 4.5초 소요됨
[홀] 모든 요리가 나왔습니다!


### (2) `await Coroutine` (코루틴을 그냥 기다릴 때)

- **행동:** "이 코루틴(함수)을 지금 당장 실행해서 끝내줘."
  
- **결과:** **제어권을 이벤트 루프에 넘기지 않습니다.** 마치 일반 함수를 호출한 것과 똑같이 행동합니다. 즉, 이 코루틴이 끝날 때까지 전체 프로그램이 멈춘 것처럼 보일 수 있습니다.

In [95]:
import asyncio

async def coro_a():
   print("나는 A야. 안녕!")

async def coro_b():
   print("나는 B야. 아무도 길 막지 말았으면...")

async def main():
   task_b = asyncio.create_task(coro_b()) # B를 예약 (하지만 아직 실행권 안 줌)

   # A를 그냥 await 함 (태스크 아님)
   # 이벤트 루프에게 턴이 넘어가지 않으므로, A가 다 끝날 때까지 B는 시작 못 함.
   for _ in range(3):
      await coro_a()

   await task_b

asyncio.run(main())

나는 A야. 안녕!
나는 A야. 안녕!
나는 A야. 안녕!
나는 B야. 아무도 길 막지 말았으면...


A가 세 번 다 실행된 후에야 B가 실행됩니다. `await coro_a()`가 이벤트 루프에게 틈을 주지 않았기 때문입니다. 만약 `await asyncio.create_task(coro_a())`로 바꾼다면, 틈이 생겨 B가 먼저 실행될 수도 있습니다.

- **참고:** `asyncio.run(main(), debug=True)`를 쓰면, 어떤 코루틴이 너무 오래 혼자 실행하고 있는지(100ms 이상) 감시해 줍니다.